# Utils

> util functions.

In [ ]:
#| default_exp utils.core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, inspect, string
from dataclasses import dataclass, field

import numpy as np, pandas as pd

from typing import List, Any, Optional, Callable, Union, Tuple, Iterable, Set, TypeAlias, Type, Dict

### Basic Utils

In [ ]:
#| export
def isiter(val: Any) -> bool:    
    return isinstance(val, Iterable)

def allinstance(vals:Any, dtype:Union[Type, TypeAlias]=Any) -> bool:
    return isiter(vals) and all(isinstance(i, dtype) for i in vals)

def allsametype(vals:Any) -> bool:
    if not isiter(vals) or len(vals) == 0: return True
    dtype = type(vals[0])
    return isiter(vals) and all(isinstance(i, dtype) for i in vals)

def isin(val:Any, vals:Iterable) -> bool:
    return val in vals

def arein(vals:Iterable, refs:Iterable) -> bool:                                             
    return isiter(vals) and isiter(refs) and all(isin(v, refs) for v in vals)

In [ ]:
#| export
def str_just_alpha(s: str) -> str:
    '''Filters a string for just alpha values'''
    return ''.join(list(filter(str.isalpha, s)))

def str_just_numeric(s: str) -> str:
    '''Filters a string for just numeric values'''
    return ''.join(list(filter(str.isnumeric, s)))

def strip_punc(s: str) -> str:
    return s.translate(str.maketrans('', '', string.punctuation))

def str_split_get(s: str, sep: str = '_', idx: int = 0) -> str:
    parts = s.split(sep)
    return parts[idx]    

def str_part_pre(s:str, sep: Optional[str] = ' ') -> str:
    pre, mid, end = s.partition(sep)
    return pre

def str_part_mid(s:str, sep: Optional[str] = ' ') -> str:
    pre, mid, end = s.partition(sep)
    return mid

def str_part_end(s:str, sep: Optional[str] = ' ') -> str:
    pre, mid, end = s.partition(sep)
    return end

### Argument and Key-Word Argument Wrangling

In [ ]:
#| export
def filter_kwargs_for_func(fn: Callable, **kwargs:Optional[dict]):
    params = inspect.signature(fn).parameters
    return {k:v for k,v in kwargs.items() if k in params}

def filter_kwargs_for_class(cls: Callable, **kwargs:Optional[dict]):
    params = inspect.signature(cls.__init__).parameters
    return {k:v for k,v in kwargs.items() if k in params}

def wrangle_kwargs_for_func(
    fn: Callable, 
    defaults: Optional[dict]=None,
    **kwargs:Optional[dict]
) -> dict:
    # copy defaults
    params = (defaults or {}).copy()
    # update with kwargs of our function
    params.update(kwargs or {})
    # filter for only the params that other function accepts
    params = filter_kwargs_for_func(fn, **params)
    return params

def wrangle_kwargs_for_class(
    cls: Callable, 
    defaults: Optional[dict]=None,
    **kwargs:Optional[dict]
) -> dict:
    # copy defaults
    params = (defaults or {}).copy()
    # update with kwargs of our class
    params.update(kwargs or {})
    # filter for only the params that other class accepts
    params = filter_kwargs_for_class(cls, **params)
    return params

def get_func_params(
    fn: Callable, 
    drop_self: Optional[bool] = True,
    drop_before: Optional[int] = 0,
    drop_idxs: Optional[List[int]] = list(),
    drop_names: Optional[List[str]] = list(),
    drop_after: Optional[int] = None,
) -> Dict[str, Any]:
    params = inspect.signature(fn).parameters
    params = {k: v.default for k, v in params.items()}
    if drop_self and 'self' in params: 
        params.pop('self')

    params = {
        n: p for i, (n, p) in enumerate(params.items()) 
        if (
            # is before <= i < after
            (drop_before <= i or (drop_after is not None and i < drop_after))
            # i not in drop_idxs and n not in drop_names
            and (i not in drop_idxs and n not in drop_names)
        )
    }  
    return params

In [ ]:
#| export
def copy_to_clipboard(text: str):
    command = ""
    
    # macOS and Linux
    if os.name == 'posix':  
        command = 'echo "' + text + '" | pbcopy'
    
    # Windows
    elif os.name == 'nt':  
        command = 'echo ' + text + ' | clip'

    else:
        raise OSError("Unsupported operating system")

    os.system(command)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()